1. Go to the [eurostat](http://ec.europa.eu/eurostat/data/database) website and try to find a dataset that includes the european unemployment rates at a recent date.

   Use this data to build a [Choropleth map](https://en.wikipedia.org/wiki/Choropleth_map) which shows the unemployment rate in Europe at a country level. Think about [the colors you use](https://carto.com/academy/courses/intermediate-design/choose-colors-1/), how you decided to [split the intervals into data classes](http://gisgeography.com/choropleth-maps-data-classification/) or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

2. Go to the [amstat](https://www.amstat.ch) website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

   > *HINT* Go to the `details` tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through. 

   Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

   The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

   Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change ? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

3. Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

4. *BONUS*: using the map you have just built, and the geographical information contained in it, could you give a *rough estimate* of the difference in unemployment rates between the areas divided by the [R繹stigraben](https://en.wikipedia.org/wiki/R%C3%B6stigraben)?

# Part3
Use the [amstat](https://www.amstat.ch) website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between *Swiss* and *foreign* workers.

   The Economic Secretary (SECO) releases [a monthly report](https://www.seco.admin.ch/seco/fr/home/Arbeit/Arbeitslosenversicherung/arbeitslosenzahlen.html) on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for *foreign* (`5.1%`) and *Swiss* (`2.2%`) workers. 

   Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (*hint* The easy way is to show two separate maps, but can you think of something better ?). Where are the differences most visible ? Why do you think that is ?

   Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.


# Import libraries used in this homework

In [1]:
import pandas as pd
import folium
import json
import os

# Create the dataframe of unemployment rate and the map of Switzerland

# Decription of dataset for unemployment
We use the dataset with the following attributes:
1. Unemployment rate indicators - Unemployment rate
2. Unemployed indicators        - Registered unemployed
3. Geographical features        - Canton
4. Other attributes             - Nationality

In [2]:
# load the dataset of unemployment rate
unemployment_data = pd.read_excel('part3_unemployment_rate_201709.xlsx')

In [3]:
unemployment_data = unemployment_data.drop(0)

In [4]:
unemployment_data['Septembre 2017'] = unemployment_data['Septembre 2017'].astype(float)


In [98]:
unemployment_data_foreigner = unemployment_data[unemployment_data['Nationalité'] == 'Etrangers']
unemployment_data_suisse = unemployment_data[unemployment_data['Nationalité'] == 'Suisses']

In [99]:
# create the map of Switzerland
suisse_coord = [46.8, 8.2]
map_suisse = folium.Map(location = suisse_coord, 
                         tiles = 'cartodbpositron',
                         zoom_start = 8)

# import the topo json file
topo_path = r'topojson/ch-cantons.topojson.json'
topo_json_data = json.load(open(topo_path))

# add the topo to the map and show the result
folium.TopoJson(open(topo_path), 'objects.cantons').add_to(map_suisse)
map_suisse

# Show the choropleth map with colors

### Choice of colormaps: diverging colormap
Reason: This kind of color map uses two more distinct colors for minimum and maximum values, which improves the drawback of sequential color map. We know sequential colormap only use one color and it's easy to make users confused. Although quantized mapping is very clear in the aspect of coloar, we think too many colors also make it difficult on understanding too.

![Sequential Colormap](sequential.png)

![Diverging Colormap](diverging.png)

![Quantized Coloarmap](quantized.png)

[Reference](http://www.kennethmoreland.com/color-maps/ColorMapsExpanded.pdf)

In [151]:
from branca.colormap import linear

colormap = linear.RdGy.scale(
    unemployment_data_foreigner['Septembre 2017'].min(),
    unemployment_data_foreigner['Septembre 2017'].max())

def printState(state):
    return colormap(unemployment_data_foreigner[unemployment_data_foreigner.Canton == state]['Septembre 2017'].iloc[0])

map_suisse = folium.Map(location = suisse_coord, 
                         tiles = 'cartodbpositron',
                         zoom_start = 8)

folium.TopoJson(topo_json_data, 
                'objects.cantons',
                style_function= lambda feature: {
                    'fillColor': printState(feature['properties']['name']),
                    'color' : 'black',
                    'weight' : 2,
                    'dashArray' : '5, 5'
                }).add_to(map_suisse)
map_suisse

# map_suisse
# unemployment_data_suisse = unemployment_data[unemployment_data.Nationalité == 'Suisses']
# map_suisse.choropleth(geo_data=open(topo_path).read(), data=unemployment_data_suisse,
#              columns=['Canton', 'Septembre 2017'],
#              key_on='objects.cantons',
#              fill_color='BuPu', fill_opacity=0.7, line_opacity=0.2,
#              legend_name='Percentage for winner of general election (%)')

In [160]:
from branca.colormap import linear
#                    'BuGn': 'Sequential',
#                    'BuPu': 'Sequential',
#                    'GnBu': 'Sequential',
#                    'OrRd': 'Sequential',
#                    'PuBu': 'Sequential',
#                    'PuBuGn': 'Sequential',
#                    'PuRd': 'Sequential',
#                    'RdPu': 'Sequential',
#                    'YlGn': 'Sequential',
#                    'YlGnBu': 'Sequential',
#                    'YlOrBr': 'Sequential',
#                    'YlOrRd': 'Sequential',
#                    'BrBg': 'Diverging',
#                    'PiYG': 'Diverging',
#                    'PRGn': 'Diverging',
#                    'PuOr': 'Diverging',
#                    'RdBu': 'Diverging',
#                    'RdGy': 'Diverging',
#                    'RdYlBu': 'Diverging',
#                    'RdYlGn': 'Diverging',
#                    'Spectral': 'Diverging',
#                    'Accent': 'Qualitative',
#                    'Dark2': 'Qualitative',
#                    'Paired': 'Qualitative',
#                    'Pastel1': 'Qualitative',
#                    'Pastel2': 'Qualitative',
#                    'Set1': 'Qualitative',
#                    'Set2': 'Qualitative',
#                    'Set3': 'Qualitative'
# YlOrRd, RdYlGn, Accent
colormap = linear.Accent.scale(
    unemployment_data_suisse['Septembre 2017'].min(),
    unemployment_data_suisse['Septembre 2017'].max())



def printState(state):
#     print(unemployment_data_suisse[unemployment_data_suisse.Canton == state]['Septembre 2017'].iloc[0])
    return colormap(unemployment_data_suisse[unemployment_data_suisse.Canton == state]['Septembre 2017'].iloc[0])

map_suisse = folium.Map(location = suisse_coord, 
                         tiles = 'cartodbpositron',
                         zoom_start = 8)

folium.TopoJson(topo_json_data, 
                'objects.cantons',
                style_function= lambda feature: {
                    'fillColor': printState(feature['properties']['name']),
                    'color' : 'black',
                    'weight' : 2,
                    'dashArray' : '5, 5'
                }).add_to(map_suisse)
# map_suisse
colormap